In [1]:
import os
os.environ['HSA_OVERRIDE_GFX_VERSION'] = '11.0.0'

gfx_version = os.getenv('HSA_OVERRIDE_GFX_VERSION')
print(f"HSA_OVERRIDE_GFX_VERSION is set to: {gfx_version}")

HSA_OVERRIDE_GFX_VERSION is set to: 11.0.0


In [2]:
!export TF_ENABLE_ONEDNN_OPTS=0

In [ ]:
#test
import torch
import os

os.environ['HSA_OVERRIDE_GFX_VERSION'] = '11.0.0'

t = torch.tensor([5, 5, 5], dtype=torch.int64, device='cuda')
t_f32 = torch.tensor([5, 5, 5], dtype=torch.float32, device='cuda')
print('Testing PyTorch ROCM support...')

print(t, t_f32)

In [8]:
!pip install pypdf  --break-system-packages -q

In [21]:
from pypdf import PdfReader
import textwrap

def load_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

In [22]:
pdf_textual_data = load_pdf("_OceanofPDF.com_Build_a_Large_Language_Model_From_Scratch_-_Sebastian_Raschka-3.pdf")
len(pdf_textual_data)

702184

In [23]:
from transformers import AutoTokenizer, AutoModel
import torch

In [24]:
model_name = "nomic-ai/nomic-embed-text-v1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [25]:
!pip install einops --break-system-packages -q

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
model

nomic-ai/nomic-bert-2048 You can inspect the repository content at https://hf.co/nomic-ai/nomic-embed-text-v1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository nomic-ai/nomic-embed-text-v1 references custom code contained in nomic-ai/nomic-bert-2048 which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/nomic-ai/nomic-bert-2048 .
 You can inspect the repository content at https://hf.co/nomic-ai/nomic-embed-text-v1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


2025-12-09 13:27:41.394600: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-09 13:27:41.793243: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/init/.local/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.30.2 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/init/

pytorch_model.bin:   0%|          | 0.00/547M [00:00<?, ?B/s]

<All keys matched successfully>


NomicBertModel(
  (embeddings): NomicBertEmbeddings(
    (word_embeddings): Embedding(30528, 768)
    (token_type_embeddings): Embedding(2, 768)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (emb_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (encoder): NomicBertEncoder(
    (layers): ModuleList(
      (0-11): 12 x NomicBertBlock(
        (attn): NomicBertAttention(
          (rotary_emb): NomicBertDynamicNTKRotaryEmbedding()
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=False)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (mlp): NomciBertGatedMLP(
          (fc11): Linear(in_features=768, out_features=3072, bias=False)
          (fc12): Linear(in_features=768, out_features=3072, bias=False)
          (fc2): Linear(in_features=3072, out_features=768, bias=False)
          (norm): Identity()
        )
        (dropout1): Dropout(p=0.0, inplace=False)
    

In [27]:
def embed(text: str):

    # Tokenize
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Forward pass
    with torch.no_grad():
        outputs = model(**tokens)

    # embeddings = mean pooling of last hidden state
    hidden_states = outputs.last_hidden_state
    attention_mask = tokens['attention_mask']

    mask = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
    emb = torch.sum(hidden_states * mask, dim=1) / torch.clamp(mask.sum(dim=1), min=1e-9)

    return emb

In [28]:
raw_embedding = embed("This is my institute")
raw_embedding.shape

torch.Size([1, 768])

In [ ]:
!pip install -q langchain langchain-community langchain-text-splitters

In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [30]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunk = splitter.split_text(pdf_textual_data)
len(text_chunk)

1510

In [ ]:
!pip install chromadb

In [31]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client()

collection = client.get_or_create_collection(
    name="pdf_chunks",
    metadata={"hnsw:space": "cosine"}  # cosine similarity for embeddings
)


In [32]:
import uuid

def embed_texts(text_list):
    vectors = []
    for text in text_list:
        v = embed(text)            # your embed() function returns tensor [1, dim]
        v = v.cpu().numpy()[0]     # convert to numpy
        vectors.append(v.tolist()) # convert to Python list
    return vectors

In [33]:
ids = [str(uuid.uuid4()) for _ in range(len(text_chunk))]
embeddings = embed_texts(text_chunk)

In [35]:
len(embeddings), len(embeddings[0])

(1510, 768)

In [36]:
collection.add(
    ids=ids,
    documents=text_chunk,
    embeddings=embeddings
)

In [37]:
query = "How the token embeddings are created in the LLM"
q_emb = embed(query).cpu().numpy()[0].tolist()

results = collection.query(
    query_embeddings=[q_emb],
    n_results=10
)

In [38]:
from pprint import pprint

for para in results["documents"]:
    pprint(para)

for para in results["distances"]:
    pprint(para)

['111 111 111 111\n'
 'Embedding of the ﬁrst token Embedding of the third token\n'
 'Figure 2.18 Positional embeddings are added to the token embedding vector to '
 'create the \n'
 'input embeddings for an LLM. The positional vectors have the same dimension '
 'as the original \n'
 'token embeddings. The token embeddings are shown with value 1 for '
 'simplicity.\n'
 '46 CHAPTER 2 Working with text data\n'
 ' Previously, we focused on very small embedding sizes for simplicity. Now, '
 'let’s con-',
 'IDs to create the token embedding vectors.\n'
 '42 CHAPTER 2 Working with text data\n'
 'these embedding weights with random values. This initialization serves as '
 'the starting\n'
 'point for the LLM’s learning process. In chapter 5, we will optimize the '
 'embedding\n'
 'weights as part of the LLM training.\n'
 ' A continuous vector representation, or embedding, is necessary since '
 'GPT-like\n'
 'LLMs are deep neural networks trained with the backpropagation algorithm.',
 'Having n